In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed

In [ ]:
%load_ext line_profiler
import line_profiler
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler import Options

directive_defaults = Options.get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [ ]:
import math
import numpy as np

def cnd(d):
    A1 = 0.31938153
    A2 = -0.356563782
    A3 = 1.781477937
    A4 = -1.821255978
    A5 = 1.330274429
    RSQRT2PI = 0.39894228040143267793994605993438
    K = 1.0 / (1.0 + 0.2316419 * np.abs(d))
    ret_val = (RSQRT2PI * np.exp(-0.5 * d * d) *
               (K * (A1 + K * (A2 + K * (A3 + K * (A4 + K * A5))))))
    if d > 0:
        return 1.0 - ret_val
    else:
        return ret_val

def blackscholes(callputflag, ulprice, strike, toexpiry, riskfreerate, iv):
    try:
        if toexpiry <= 0:
            # Calculate payoff at expiration
            if callputflag == 'C' or callputflag == 'c':
                return max(0.0, ulprice - strike)
            else:
                return max(0.0, strike - ulprice)

        d1 = (math.log(ulprice / strike) + (riskfreerate + iv * iv / 2) * toexpiry) / (iv * math.sqrt(toexpiry))
        d2 = d1 - iv * math.sqrt(toexpiry)

        if callputflag == 'C' or callputflag == 'c':
            bsPrice = ulprice * cnd(d1) - strike * math.exp(-riskfreerate * toexpiry) * cnd(d2)
        else:
            bsPrice = strike * math.exp(-riskfreerate * toexpiry) * cnd(-d2) - ulprice * cnd(-d1)
        return bsPrice
    except:
        return 0.0

def blackscholes_greeks(callputflag, ulprice, strike, toexpiry, riskfreerate, iv):
    try:
        if toexpiry <= 0:
            # Calculate greeks at expiration
            if callputflag == 'C' or callputflag == 'c':
                delta = 1.0 if ulprice > strike else 0.0
            else:
                delta = -1.0 if ulprice < strike else 0.0
            return (delta, )

        d1 = (math.log(ulprice / strike) + (riskfreerate + iv * iv / 2) * toexpiry) / (iv * math.sqrt(toexpiry))
        d2 = d1 - iv * math.sqrt(toexpiry)
        if callputflag == 'C' or callputflag == 'c':
            # Call greeks
            call_delta = cnd(d1)
            return (call_delta, )
        else:
            # put greeks
            put_delta = -cnd(-d1)
            return (put_delta, )
    except:
        return (0.0,)

In [ ]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
from libc.math cimport exp, log, sqrt, abs
import numpy as np
import cython

@cython.cdivision(True)
cdef float cnd(float d):
    cdef float A1 = 0.31938153
    cdef float A2 = -0.356563782
    cdef float A3 = 1.781477937
    cdef float A4 = -1.821255978
    cdef float A5 = 1.330274429
    cdef float RSQRT2PI = 0.39894228040143267793994605993438
    cdef float K = 1.0 / (1.0 + 0.2316419 * abs(d))
    cdef float ret_val = (RSQRT2PI * exp(-0.5 * d * d) *
               (K * (A1 + K * (A2 + K * (A3 + K * (A4 + K * A5))))))
    if d > 0:
        return 1.0 - ret_val
    else:
        return ret_val

@cython.cdivision(True)
def blackscholes_fast(int iscall,float ulprice, float strike, float toexpiry, float riskfreerate,float iv):
    cdef float  bsPrice = 0.0
    cdef float d1
    cdef float d2
    if toexpiry <= 0:
        # Calculate payoff at expiration
        if iscall == 1:
            return max(0.0, ulprice - strike)
        else:
            return max(0.0, strike - ulprice)

    d1 = (log(ulprice / strike) + (riskfreerate + iv * iv / 2) * toexpiry) / (iv * sqrt(toexpiry))
    d2 = d1 - iv * sqrt(toexpiry)

    if iscall == 1:
        bsPrice = ulprice * cnd(d1) - strike * exp(-riskfreerate * toexpiry) * cnd(d2)
    else:
        bsPrice = strike * exp(-riskfreerate * toexpiry) * cnd(-d2) - ulprice * cnd(-d1)
    return bsPrice

@cython.cdivision(True)
def blackscholes_greeks(int iscall,float ulprice, float strike, float toexpiry, float riskfreerate,float iv):
    if toexpiry <= 0:
        # Calculate greeks at expiration
        if iscall == 1:
            delta = 1.0 if ulprice > strike else 0.0
        else:
            delta = -1.0 if ulprice < strike else 0.0
        return (delta, )

    d1 = (log(ulprice / strike) + (riskfreerate + iv * iv / 2) * toexpiry) / (iv * sqrt(toexpiry))
    d2 = d1 - iv * sqrt(toexpiry)
    if iscall == 1:
        # Call greeks
        call_delta = cnd(d1)
        return (call_delta, )
    else:
        # put greeks
        put_delta = -cnd(-d1)
        return (put_delta, )

In [ ]:
%lprun -f get_quote_fast get_quote_fast(DataFrameGetter(df), [tz.localize(pd.Timestamp('2012-07-15 16:59:00.000'))])

In [ ]:
%timeit blackscholes('C', 100.0, 100.0, 0.24, 0.0, 0.3)

In [ ]:
%timeit blackscholes_fast(1, 100.0, 100.0, 0.24, 0.0, 0.3)

In [ ]:
%lprun -f blackscholes_fast blackscholes_fast(1, 100.0, 100.0, 0.24, 0.0, 0.3)